## Part 2: Data Cleaning and Processing

In [1]:
import numpy as np
import pandas as pd

In [2]:
base_dir = "/Users/koushalsmodi/Desktop/MachineLearning/MachineLearningProjects/TensorFlow/CreditRisk"
df = pd.read_csv(base_dir+"/home-credit-default-risk/application_train.csv")

In [3]:
print("="* 60)
print("Data Cleaning and Processing")
print("="* 60)

Data Cleaning and Processing


In [4]:
def clean_credit_data(df):
    """ Cleaning data for use in model training:
    1: Handling Missing Values
    2: Handling Outliers
    3: Remove Duplicates
    4: Data Consistency Checks
    """
    
    df_clean = df.copy()
    cleaning_report = []
    
    print("\n Step 1: Handling Missing Values")
    
    # I am looking at patterns for missing values
    
    missing_cols = df_clean.columns[df_clean.isnull().any()].tolist()
    
    for col in missing_cols:
        missing = df_clean[col].isnull().sum()
        missing_pct = 100 * missing / len(df_clean)
        

        if missing_pct > 50:
            # Dropping columns if > 50% is missing
            df_clean = df_clean.drop(col, axis = 1)
            cleaning_report.append(f"Dropped {col}: {missing_pct: .2f}% missing")
            
        elif df_clean[col].dtype in ['int64', 'float64']:
            # Imputing numerical value with median (as it is not affected by outliers)
            median_val = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(median_val)
            cleaning_report.append(f"Imputed {col} with median: {median_val:.2f}")
            
        else:
            # Mode for categorical values imputation
            mode_val = df_clean[col].mode()[0] if len(df_clean[col].mode()) > 0 else 'Unknown'
            df_clean[col] = df_clean[col].fillna(mode_val)
            cleaning_report.append(f"Imputed {col} with mode: {mode_val}")
        
    print(f"Processed {len(missing_cols)} columns with missing values")
    
    print("\n Step 2: Handling Outliers")
    
    numeric_cols = df_clean.select_dtypes(include = [np.number]).columns.tolist()
    numeric_cols = [col for col in numeric_cols if col != 'TARGET']
    
    outlier_summary = []
    
     # Taking only first few to be efficient when running
    for col in numeric_cols[:6]:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        
        IQR = Q3 - Q1 
        
        # lower bound and upper bound
        lower_bound = Q1 - 1.5 * IQR 
        upper_bound = Q3 + 1.5 * IQR 
        
        if col.startswith('AMT_') or col.startswith('CNT_'):
            lower_bound = max(0, lower_bound)
        
        if 'AGE' in col:
            lower_bound = max(0, lower_bound)
            upper_bound = min(100, upper_bound)
        
        # Counting outliers
        
        outliers = ((df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)).sum()
        
        if outliers > 0:
            outlier_pct = 100 * outliers / len(df_clean)
            
            # Capping by clipping outliers instead of removing 
            df_clean[col] = df_clean[col].clip(lower = lower_bound, upper = upper_bound)
            
            outlier_summary.append({
                "Feature": col,
                "Outliers": outliers,
                "Percentage": outlier_pct,
                "Lower Bound": lower_bound,
                "Upper Bound": upper_bound
            })
            
    if outlier_summary:
        outlier_df = pd.DataFrame(outlier_summary)
        print("\n Outliers Detected and Capped")
        print(outlier_df.to_string())
        
    
    print("\n Step 3: Remove Duplicates")
    
    initial_rows = len(df_clean)
    df_clean.drop_duplicates(inplace = True)
    duplicates_removed = initial_rows - len(df_clean)
    
    print(f"Duplicates removed: {duplicates_removed}")
    
    print("\n Step 4: Data Consistency Checks")
    
    issues_found = 0
    
    if 'AMT_INCOME_TOTAL' in df_clean.columns:
        # Keeping only records where income >= 0
        invalid = (df_clean['AMT_INCOME_TOTAL'] < 0).sum()
        
        if invalid > 0:
            df_clean = df_clean[df_clean['AMT_INCOME_TOTAL'] >= 0]
            print(f"Removed {invalid} records with invalid income")
            issues_found += 1 
    
    print(f"\n Total issues found and corrected: {issues_found}")
    
    print("\n" + "=" * 60)
    print("Cleaning Summary")
    print("\n" + "=" * 60)
    
    print(f"\n Original Shape: {df.shape}")
    print(f"\n Cleaned Shape: {df_clean.shape}")
    
    print(f"Rows removed: {df.shape[0] - df_clean.shape[0]} ({100 * (df.shape[0] - df_clean.shape[0])/df.shape[0]:.2f}%)")
    print(f"Columns removed: {df.shape[1] - df_clean.shape[1]}")
        
    
    return df_clean

In [5]:
df_cleaned = clean_credit_data(df)


 Step 1: Handling Missing Values
Processed 67 columns with missing values

 Step 2: Handling Outliers

 Outliers Detected and Capped
            Feature  Outliers  Percentage  Lower Bound  Upper Bound
0      CNT_CHILDREN      4272    1.389219            0          2.5
1  AMT_INCOME_TOTAL     14035    4.564064            0     337500.0
2        AMT_CREDIT      6562    2.133907            0    1616625.0
3       AMT_ANNUITY      7504    2.440238            0      61704.0
4   AMT_GOODS_PRICE     14728    4.789422            0    1341000.0

 Step 3: Remove Duplicates
Duplicates removed: 0

 Step 4: Data Consistency Checks

 Total issues found and corrected: 0

Cleaning Summary


 Original Shape: (307511, 122)

 Cleaned Shape: (307511, 81)
Rows removed: 0 (0.00%)
Columns removed: 41


In [6]:
df_clean = clean_credit_data(df)

df_clean.to_csv('application_train_cleaned.csv', index=False)
print("✓ Saved cleaned data to application_train_cleaned.csv")


 Step 1: Handling Missing Values
Processed 67 columns with missing values

 Step 2: Handling Outliers

 Outliers Detected and Capped
            Feature  Outliers  Percentage  Lower Bound  Upper Bound
0      CNT_CHILDREN      4272    1.389219            0          2.5
1  AMT_INCOME_TOTAL     14035    4.564064            0     337500.0
2        AMT_CREDIT      6562    2.133907            0    1616625.0
3       AMT_ANNUITY      7504    2.440238            0      61704.0
4   AMT_GOODS_PRICE     14728    4.789422            0    1341000.0

 Step 3: Remove Duplicates
Duplicates removed: 0

 Step 4: Data Consistency Checks

 Total issues found and corrected: 0

Cleaning Summary


 Original Shape: (307511, 122)

 Cleaned Shape: (307511, 81)
Rows removed: 0 (0.00%)
Columns removed: 41
✓ Saved cleaned data to application_train_cleaned.csv


In [7]:
# Next is univariate analysis